# This Notebook contains code for web Scraping. 

It is Scraping Daraz Product reviews up till 5th page for 200 products till 5th main page. It can used for extrating product reviews from any market place by adjusting parameters according to web page HTML structure.

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

def extract_title_and_reviews():
    global max_review_pages
    temp = {'Name': [], 'Review text': []}
    
    try:
        title_element = driver.find_element(By.CSS_SELECTOR, '#module_product_title_1 > div > div > span').text
    except NoSuchElementException:
        print("Title not found")
        return None
    
    driver.execute_script("window.scrollTo(0, 500);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(500, 800);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(800, 1000);")
    time.sleep(2)
    
    next_button_clicks = 0 

    for page in range(max_review_pages):
        while True:
            try:
                reviews_container = driver.find_element(By.CSS_SELECTOR, '#module_product_review > div > div > div.review-content')
            except NoSuchElementException:
                print('No Reviews')
                break
            reviews = reviews_container.find_elements(By.CLASS_NAME,'review-item')

            # Check if there are no reviews on the page
            if not reviews:
                print('No Reviews on this page')
                break

            for i in range(len(reviews)):
                if reviews[i].find_element(By.CSS_SELECTOR, f'#module_product_review > div > div > div.review-content > div:nth-child({i+1}) > div.review-content-sl').text != '':
                    temp['Name'].append(reviews[i].find_element(By.CLASS_NAME,'user').text[3:])
                    temp['Review text'].append(reviews[i].find_element(By.CSS_SELECTOR, f"#module_product_review > div > div > div.review-content > div:nth-child({i+1}) > div.review-content-sl").text)

            try:
                if(page != max_review_pages - 1 and next_button_clicks < max_review_pages - 1):
                    next_button = driver.find_element(By.CSS_SELECTOR, '#module_product_review .ant-pagination-next button')
                    if not next_button.is_enabled():
                        break  # No more pages, exit the loop
                    else:
                        driver.execute_script("arguments[0].click();", next_button)
                        time.sleep(2)  
                        next_button_clicks += 1
                else:
                    print("No More Pages or Maximum Pages Reached")
                    break
            except NoSuchElementException:
                print('No More Pages')
                break
            except Exception as e:
                print(f"An error occurred: {e}")
                break


    df_combined = pd.DataFrame({
        'Title': [title_element] * len(temp['Name']),
        'Name': temp['Name'],
        'Review text': temp['Review text']
    })

    return df_combined
    

driver.get('https://www.daraz.pk/catalog/?q=mobile+phones&_keyori=ss&from=input&spm=a2a0e.searchlist.search.go.6ded431bvmtNaX')
time.sleep(3)
columns = ["Title", "Name", "Review text"]
df = pd.DataFrame(columns=columns)

num_pages = 6
products_per_page = 40
max_review_pages = 5



try:
    for page in range(num_pages):
        print(f"Extracting data from page {page + 1}...")

        product_elements = driver.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')

        for index, product_element in enumerate(product_elements):
            try:
                print(f"Extracting data for product {index + 1} on page {page + 1}...")

                product_url_element = product_element.find_element(By.TAG_NAME, 'a')
                if product_url_element:
                    product_url = product_url_element.get_attribute('href')

                    # Open the product URL in a new tab or window
                    driver.execute_script("window.open('', '_blank');")
                    driver.switch_to.window(driver.window_handles[1])
                    driver.get(product_url)

                    try:
                        product_info = extract_title_and_reviews()

                        # Check if 'Review text' column is not empty
                        if not product_info['Review text'].empty:
                            
                            for review_index, review_row in product_info.iterrows():
                                row_data = pd.Series({
                                    'Title': review_row['Title'],
                                    'Name': review_row['Name'],
                                    'Review text': review_row['Review text']
                                })
                                df = df.append(row_data, ignore_index=True)
                        else:
                            print("No review text available for this product. Skipping.")
                    except NoSuchElementException:
                        print("Error extracting product info. Skipping.")
                    finally:
                        # Close the tab or window
                        driver.close()
                        # Switch back to the main page
                        driver.switch_to.window(driver.window_handles[0])

            except NoSuchElementException:
                print("Product URL not found. Skipping.")
            except Exception as e:
                print(f"An error occurred: {e}")

        
        if page < num_pages - 1:  # Updated condition to ensure it stops at the last page
            try:
                driver.execute_script("window.scrollTo(0, 3850);")
                time.sleep(10)
                page_number = page + 2  
                print(f"Clicking on page {page_number} button...")
                page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, f'ant-pagination-item-{page_number}'))
                )
                page_button.click()
                time.sleep(2)  # Adding a delay to ensure the page is loaded before continuing
            except NoSuchElementException:
                print(f"Page button for page {page_number} not found. Exiting.")
                break
            except Exception as e:
                print(f"An error occurred: {e}")
                break
finally:
    # Close the browser
    driver.quit()

    print(df)

Extracting data from page 1...
Extracting data for product 1 on page 1...
No More Pages or Maximum Pages Reached
No More Pages or Maximum Pages Reached
No More Pages or Maximum Pages Reached
No More Pages or Maximum Pages Reached
No More Pages or Maximum Pages Reached


C:\Users\HP\AppData\Local\Temp\ipykernel_13792\972393118.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_13792\972393118.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_13792\972393118.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_13792\972393118.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\HP\AppData\Loca

Extracting data for product 2 on page 1...
No More Pages or Maximum Pages Reached


C:\Users\HP\AppData\Local\Temp\ipykernel_13792\972393118.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_13792\972393118.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_13792\972393118.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_13792\972393118.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\HP\AppData\Loca

Extracting data for product 3 on page 1...
No More Pages or Maximum Pages Reached
No More Pages or Maximum Pages Reached
No More Pages or Maximum Pages Reached
No More Pages or Maximum Pages Reached
No More Pages or Maximum Pages Reached


...

Extracting data for product 39 on page 6...
No More Pages or Maximum Pages Reached


C:\Users\HP\AppData\Local\Temp\ipykernel_13792\972393118.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_13792\972393118.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_13792\972393118.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_13792\972393118.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)
C:\Users\HP\AppData\Loca

Extracting data for product 40 on page 6...
No More Pages or Maximum Pages Reached


C:\Users\HP\AppData\Local\Temp\ipykernel_13792\972393118.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_data, ignore_index=True)


                                                  Title      Name  \
0     Redmi Note 12 (8GB-128GB) PTA Approved With Of...    eer H.   
1     Redmi Note 12 (8GB-128GB) PTA Approved With Of...   ****413   
2     Redmi Note 12 (8GB-128GB) PTA Approved With Of...     el A.   
3     Redmi Note 12 (8GB-128GB) PTA Approved With Of...      d R.   
4     Redmi Note 12 (8GB-128GB) PTA Approved With Of...  ammad J.   
...                                                 ...       ...   
2959  Infinix Hot 30 RAM 8 GB ROM 128 GB Back Camera...  ammad N.   
2960  Infinix Hot 30 RAM 8 GB ROM 128 GB Back Camera...  ammad N.   
2961  Infinix Hot 30 RAM 8 GB ROM 128 GB Back Camera...  ammad N.   
2962  Infinix Hot 30 RAM 8 GB ROM 128 GB Back Camera...  ammad N.   
2963  Samsung Galaxy A04 - 6.5" Inch Display - 4GB R...  ammad I.   

                                            Review text  
0     This is really amazing, such a beautiful colou...  
1     nice set and same colour as orderd. premium to...

In [3]:
import pandas as pd
df=pd.read_csv('reviews.csv')
df

Title      Name  \
0     Redmi Note 12 (8GB-128GB) PTA Approved With Of...    eer H.   
1     Redmi Note 12 (8GB-128GB) PTA Approved With Of...   ****413   
2     Redmi Note 12 (8GB-128GB) PTA Approved With Of...     el A.   
3     Redmi Note 12 (8GB-128GB) PTA Approved With Of...      d R.   
4     Redmi Note 12 (8GB-128GB) PTA Approved With Of...  ammad J.   
...                                                 ...       ...   
2959  Infinix Hot 30 RAM 8 GB ROM 128 GB Back Camera...  ammad N.   
2960  Infinix Hot 30 RAM 8 GB ROM 128 GB Back Camera...  ammad N.   
2961  Infinix Hot 30 RAM 8 GB ROM 128 GB Back Camera...  ammad N.   
2962  Infinix Hot 30 RAM 8 GB ROM 128 GB Back Camera...  ammad N.   
2963  Samsung Galaxy A04 - 6.5" Inch Display - 4GB R...  ammad I.   

                                            Review text  
0     This is really amazing, such a beautiful colou...  
1     nice set and same colour as orderd. premium to...  
2     I got this mobile on time and it is a good mob...  
3     I ordered phone from Daraz, received as shown....  
4     #11.11 "I recently purchased a mobile from Dar...  
...                                                 ...  
2959  Excellent service, 1 din main mobile phone rec...  
2960  Excellent service, 1 din main mobile phone rec...  
2961  Excellent service, 1 din main mobile phone rec...  
2962  Excellent service, 1 din main mobile phone rec...  
2963                      very good I like it very much  

[2964 rows x 3 columns]

In [4]:
df.drop_duplicates(inplace=True)
print(df)

                                                  Title      Name  \
0     Redmi Note 12 (8GB-128GB) PTA Approved With Of...    eer H.   
1     Redmi Note 12 (8GB-128GB) PTA Approved With Of...   ****413   
2     Redmi Note 12 (8GB-128GB) PTA Approved With Of...     el A.   
3     Redmi Note 12 (8GB-128GB) PTA Approved With Of...      d R.   
4     Redmi Note 12 (8GB-128GB) PTA Approved With Of...  ammad J.   
...                                                 ...       ...   
2954  Tecno Spark 10C || 4GB Ram 128GB Rom || 5000 m...   boob A.   
2955  Tecno Spark 10C || 4GB Ram 128GB Rom || 5000 m...     ad L.   
2956  Tecno Spark 10C || 4GB Ram 128GB Rom || 5000 m...        H.   
2958  Infinix Hot 30 RAM 8 GB ROM 128 GB Back Camera...  ammad N.   
2963  Samsung Galaxy A04 - 6.5" Inch Display - 4GB R...  ammad I.   

                                            Review text  
0     This is really amazing, such a beautiful colou...  
1     nice set and same colour as orderd. premium to...

In [5]:
df

Title      Name  \
0     Redmi Note 12 (8GB-128GB) PTA Approved With Of...    eer H.   
1     Redmi Note 12 (8GB-128GB) PTA Approved With Of...   ****413   
2     Redmi Note 12 (8GB-128GB) PTA Approved With Of...     el A.   
3     Redmi Note 12 (8GB-128GB) PTA Approved With Of...      d R.   
4     Redmi Note 12 (8GB-128GB) PTA Approved With Of...  ammad J.   
...                                                 ...       ...   
2954  Tecno Spark 10C || 4GB Ram 128GB Rom || 5000 m...   boob A.   
2955  Tecno Spark 10C || 4GB Ram 128GB Rom || 5000 m...     ad L.   
2956  Tecno Spark 10C || 4GB Ram 128GB Rom || 5000 m...        H.   
2958  Infinix Hot 30 RAM 8 GB ROM 128 GB Back Camera...  ammad N.   
2963  Samsung Galaxy A04 - 6.5" Inch Display - 4GB R...  ammad I.   

                                            Review text  
0     This is really amazing, such a beautiful colou...  
1     nice set and same colour as orderd. premium to...  
2     I got this mobile on time and it is a good mob...  
3     I ordered phone from Daraz, received as shown....  
4     #11.11 "I recently purchased a mobile from Dar...  
...                                                 ...  
2954                                               Good  
2955                                               Good  
2956                                  AMAZING MOBILE ❣️  
2958  Excellent service, 1 din main mobile phone rec...  
2963                      very good I like it very much  

[1145 rows x 3 columns]

In [6]:
df.to_csv("reviews_duplicates_removed.csv",index=False)

In [1]:
import pandas as pd
df=pd.read_csv("reviews_duplicates_removed.csv")
keywords_to_exclude = ['case', 'cover', 'charger', 'cable']
df_filtered = df[~df['Title'].str.lower().str.contains('|'.join(keywords_to_exclude).lower(), na=False)].copy()
df_filtered

Title      Name  \
0     Redmi Note 12 (8GB-128GB) PTA Approved With Of...    eer H.   
1     Redmi Note 12 (8GB-128GB) PTA Approved With Of...   ****413   
2     Redmi Note 12 (8GB-128GB) PTA Approved With Of...     el A.   
3     Redmi Note 12 (8GB-128GB) PTA Approved With Of...      d R.   
4     Redmi Note 12 (8GB-128GB) PTA Approved With Of...  ammad J.   
...                                                 ...       ...   
1140  Tecno Spark 10C || 4GB Ram 128GB Rom || 5000 m...   boob A.   
1141  Tecno Spark 10C || 4GB Ram 128GB Rom || 5000 m...     ad L.   
1142  Tecno Spark 10C || 4GB Ram 128GB Rom || 5000 m...        H.   
1143  Infinix Hot 30 RAM 8 GB ROM 128 GB Back Camera...  ammad N.   
1144  Samsung Galaxy A04 - 6.5" Inch Display - 4GB R...  ammad I.   

                                            Review text  
0     This is really amazing, such a beautiful colou...  
1     nice set and same colour as orderd. premium to...  
2     I got this mobile on time and it is a good mob...  
3     I ordered phone from Daraz, received as shown....  
4     #11.11 "I recently purchased a mobile from Dar...  
...                                                 ...  
1140                                               Good  
1141                                               Good  
1142                                  AMAZING MOBILE ❣️  
1143  Excellent service, 1 din main mobile phone rec...  
1144                      very good I like it very much  

[1115 rows x 3 columns]

In [2]:
df_filtered.to_csv('reviews_filtered.csv',index=False)

In [4]:
df=df_filtered
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

def sentiment(review):
    analyser=SentimentIntensityAnalyzer()
    polarity=analyser.polarity_scores(review)['compound']
    
    if polarity >= 0.05:
        return 'Positive'
    elif polarity <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'
df["Sentiment"]=df["Review text"].apply(sentiment)

df
    
        

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Title      Name  \
0     Redmi Note 12 (8GB-128GB) PTA Approved With Of...    eer H.   
1     Redmi Note 12 (8GB-128GB) PTA Approved With Of...   ****413   
2     Redmi Note 12 (8GB-128GB) PTA Approved With Of...     el A.   
3     Redmi Note 12 (8GB-128GB) PTA Approved With Of...      d R.   
4     Redmi Note 12 (8GB-128GB) PTA Approved With Of...  ammad J.   
...                                                 ...       ...   
1140  Tecno Spark 10C || 4GB Ram 128GB Rom || 5000 m...   boob A.   
1141  Tecno Spark 10C || 4GB Ram 128GB Rom || 5000 m...     ad L.   
1142  Tecno Spark 10C || 4GB Ram 128GB Rom || 5000 m...        H.   
1143  Infinix Hot 30 RAM 8 GB ROM 128 GB Back Camera...  ammad N.   
1144  Samsung Galaxy A04 - 6.5" Inch Display - 4GB R...  ammad I.   

                                            Review text Sentiment  
0     This is really amazing, such a beautiful colou...  Positive  
1     nice set and same colour as orderd. premium to...  Positive  
2     I got this mobile on time and it is a good mob...  Positive  
3     I ordered phone from Daraz, received as shown....  Negative  
4     #11.11 "I recently purchased a mobile from Dar...  Positive  
...                                                 ...       ...  
1140                                               Good  Positive  
1141                                               Good  Positive  
1142                                  AMAZING MOBILE ❣️  Positive  
1143  Excellent service, 1 din main mobile phone rec...  Positive  
1144                      very good I like it very much  Positive  

[1115 rows x 4 columns]

In [5]:
sentiment_to_score = {'Positive': 5, 'Neutral': 3, 'Negative': 1}
df['Review_Score'] = df['Sentiment'].map(sentiment_to_score)
df

Title      Name  \
0     Redmi Note 12 (8GB-128GB) PTA Approved With Of...    eer H.   
1     Redmi Note 12 (8GB-128GB) PTA Approved With Of...   ****413   
2     Redmi Note 12 (8GB-128GB) PTA Approved With Of...     el A.   
3     Redmi Note 12 (8GB-128GB) PTA Approved With Of...      d R.   
4     Redmi Note 12 (8GB-128GB) PTA Approved With Of...  ammad J.   
...                                                 ...       ...   
1140  Tecno Spark 10C || 4GB Ram 128GB Rom || 5000 m...   boob A.   
1141  Tecno Spark 10C || 4GB Ram 128GB Rom || 5000 m...     ad L.   
1142  Tecno Spark 10C || 4GB Ram 128GB Rom || 5000 m...        H.   
1143  Infinix Hot 30 RAM 8 GB ROM 128 GB Back Camera...  ammad N.   
1144  Samsung Galaxy A04 - 6.5" Inch Display - 4GB R...  ammad I.   

                                            Review text Sentiment  \
0     This is really amazing, such a beautiful colou...  Positive   
1     nice set and same colour as orderd. premium to...  Positive   
2     I got this mobile on time and it is a good mob...  Positive   
3     I ordered phone from Daraz, received as shown....  Negative   
4     #11.11 "I recently purchased a mobile from Dar...  Positive   
...                                                 ...       ...   
1140                                               Good  Positive   
1141                                               Good  Positive   
1142                                  AMAZING MOBILE ❣️  Positive   
1143  Excellent service, 1 din main mobile phone rec...  Positive   
1144                      very good I like it very much  Positive   

      Review_Score  
0                5  
1                5  
2                5  
3                1  
4                5  
...            ...  
1140             5  
1141             5  
1142             5  
1143             5  
1144             5  

[1115 rows x 5 columns]

In [6]:
df=df.drop("Sentiment",axis=1)
df

Title      Name  \
0     Redmi Note 12 (8GB-128GB) PTA Approved With Of...    eer H.   
1     Redmi Note 12 (8GB-128GB) PTA Approved With Of...   ****413   
2     Redmi Note 12 (8GB-128GB) PTA Approved With Of...     el A.   
3     Redmi Note 12 (8GB-128GB) PTA Approved With Of...      d R.   
4     Redmi Note 12 (8GB-128GB) PTA Approved With Of...  ammad J.   
...                                                 ...       ...   
1140  Tecno Spark 10C || 4GB Ram 128GB Rom || 5000 m...   boob A.   
1141  Tecno Spark 10C || 4GB Ram 128GB Rom || 5000 m...     ad L.   
1142  Tecno Spark 10C || 4GB Ram 128GB Rom || 5000 m...        H.   
1143  Infinix Hot 30 RAM 8 GB ROM 128 GB Back Camera...  ammad N.   
1144  Samsung Galaxy A04 - 6.5" Inch Display - 4GB R...  ammad I.   

                                            Review text  Review_Score  
0     This is really amazing, such a beautiful colou...             5  
1     nice set and same colour as orderd. premium to...             5  
2     I got this mobile on time and it is a good mob...             5  
3     I ordered phone from Daraz, received as shown....             1  
4     #11.11 "I recently purchased a mobile from Dar...             5  
...                                                 ...           ...  
1140                                               Good             5  
1141                                               Good             5  
1142                                  AMAZING MOBILE ❣️             5  
1143  Excellent service, 1 din main mobile phone rec...             5  
1144                      very good I like it very much             5  

[1115 rows x 4 columns]

In [7]:
df.to_csv('reviews_filtered.csv',index=False)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1115 entries, 0 to 1144
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         1115 non-null   object
 1   Name          1111 non-null   object
 2   Review text   1115 non-null   object
 3   Review_Score  1115 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 43.6+ KB
